In [1]:
from dateutil import parser
import requests
import pandas as pd
import json


In [2]:
API_KEY = 'eec6ddcc5fa2bc2115c3a9ae1069c5da-183f9c7dcc76b62a9a77824419b8538a'
ACCOUNT_ID = '101-001-28743255-001'
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

In [3]:
session = requests.Session()

In [4]:
session.headers.update({
    'Authorization':  f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
})

In [5]:
params = {
    'count': 10,
    'granularity': 'H1',
    'price': 'MBA'
}

In [6]:
url = f'{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments'

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
response.status_code

200

In [9]:
data = response.json()

In [10]:
instruments_list = data['instruments']

In [11]:
len(instruments_list)

68

In [12]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [13]:
instruments_keys = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [14]:
instruments_dict = {}

for instr in instruments_list:
    instruments_dict[instr['name']] = {key:instr[key] for key in instruments_keys}


In [15]:
instruments_dict['GBP_USD']

{'name': 'GBP_USD',
 'type': 'CURRENCY',
 'displayName': 'GBP/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.05'}

In [16]:
with open('../data/instruments.json', 'w') as file:
    file.write(json.dumps(instruments_dict, indent=2))
    

In [26]:
def fetch_candles(pair, count=10, granularity='H1'):
    url = f'{OANDA_URL}/instruments/{pair}/candles'
    
    params = {
        'count': count,
        'granularity': granularity,
        'price': 'MBA'
    }

    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']

    return response.status_code, data


In [27]:
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    final_data = []
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    for candle in data:
        new_dict = {}
    
        if candle['complete'] is True:
            new_dict['time'] = parser.parse(candle['time'])
            new_dict['volume'] = candle['volume']
    
            for price in prices:
                for o in ohlc:
                    new_dict[f'{price}_{o}'] = float(candle[price][o])
            
            final_data.append(new_dict)
        else:
            continue
        
    final_data_df = pd.DataFrame.from_dict(data=final_data)
    return final_data_df


In [42]:
def create_data_file(pair, count=10, granularity='H1'):
    status_code, data = fetch_candles(pair, count, granularity)

    assert status_code == 200, f'Failed {pair} - {data}'
    assert len(data) > 0, f'No candles {pair}'

    candles_df = get_candles_df(data)
    candles_df.to_pickle(f'../data/{pair}_{granularity}.pkl')
    
    print(f'{pair} - {granularity} - {candles_df.shape[0]} candles - {candles_df['time'].min()} - {candles_df['time'].max()}')


In [28]:
status_code, data = fetch_candles('GBP_USD', count=7, granularity='H4')

In [33]:
status_code

200

In [32]:
candles_df = get_candles_df(data)

In [30]:
len(data)

7

In [31]:
data[0]

{'complete': True,
 'volume': 10886,
 'time': '2024-05-28T01:00:00.000000000Z',
 'bid': {'o': '1.27701', 'h': '1.27826', 'l': '1.27695', 'c': '1.27740'},
 'mid': {'o': '1.27710', 'h': '1.27835', 'l': '1.27704', 'c': '1.27748'},
 'ask': {'o': '1.27719', 'h': '1.27844', 'l': '1.27714', 'c': '1.27755'}}

In [34]:
candles_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    6 non-null      datetime64[ns, tzutc()]
 1   volume  6 non-null      int64                  
 2   mid_o   6 non-null      float64                
 3   mid_h   6 non-null      float64                
 4   mid_l   6 non-null      float64                
 5   mid_c   6 non-null      float64                
 6   bid_o   6 non-null      float64                
 7   bid_h   6 non-null      float64                
 8   bid_l   6 non-null      float64                
 9   bid_c   6 non-null      float64                
 10  ask_o   6 non-null      float64                
 11  ask_h   6 non-null      float64                
 12  ask_l   6 non-null      float64                
 13  ask_c   6 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in

In [35]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-05-28 01:00:00+00:00,10886,1.27710,1.27835,1.27704,1.27748,1.27701,1.27826,1.27695,1.27740,1.27719,1.27844,1.27714,1.27755
1,2024-05-28 05:00:00+00:00,13138,1.27747,1.27812,1.27626,1.27690,1.27738,1.27804,1.27618,1.27681,1.27756,1.27820,1.27634,1.27698
2,2024-05-28 09:00:00+00:00,14400,1.27689,1.27948,1.27638,1.27924,1.27681,1.27938,1.27628,1.27916,1.27697,1.27957,1.27647,1.27931
3,2024-05-28 13:00:00+00:00,17628,1.27922,1.28008,1.27708,1.27766,1.27914,1.28001,1.27700,1.27758,1.27931,1.28016,1.27715,1.27774
4,2024-05-28 17:00:00+00:00,12899,1.27766,1.27782,1.27541,1.27619,1.27758,1.27773,1.27532,1.27610,1.27773,1.27790,1.27550,1.27628
5,2024-05-28 21:00:00+00:00,7012,1.27602,1.27660,1.27522,1.27542,1.27565,1.27632,1.27495,1.27534,1.27640,1.27731,1.27529,1.27549


In [43]:
create_data_file('GBP_USD', count=7, granularity='H4')

GBP_USD - H4 - 6 candles - 2024-05-28 01:00:00+00:00 - 2024-05-28 21:00:00+00:00


In [45]:
currencies = ['GBP', 'USD', 'CHF', 'AUD', 'SGD', 'NOK', 'EUR']

In [48]:
for curr_1 in currencies:
    for curr_2 in currencies:
        pair = f'{curr_1}_{curr_2}'
        
        if pair in instruments_dict:
            for gran in ['H1', 'H4']:
                create_data_file(pair=pair, count=5000, granularity=gran)

GBP_USD - H1 - 4999 candles - 2023-08-08 17:00:00+00:00 - 2024-05-29 01:00:00+00:00
GBP_USD - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:00+00:00
GBP_CHF - H1 - 4999 candles - 2023-08-08 17:00:00+00:00 - 2024-05-29 01:00:00+00:00
GBP_CHF - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:00+00:00
GBP_AUD - H1 - 4999 candles - 2023-08-08 17:00:00+00:00 - 2024-05-29 01:00:00+00:00
GBP_AUD - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:00+00:00
GBP_SGD - H1 - 4999 candles - 2023-08-08 14:00:00+00:00 - 2024-05-29 01:00:00+00:00
GBP_SGD - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:00+00:00
USD_CHF - H1 - 4999 candles - 2023-08-08 17:00:00+00:00 - 2024-05-29 01:00:00+00:00
USD_CHF - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:00+00:00
USD_SGD - H1 - 4999 candles - 2023-08-08 17:00:00+00:00 - 2024-05-29 01:00:00+00:00
USD_SGD - H4 - 4999 candles - 2021-03-15 21:00:00+00:00 - 2024-05-28 21:00:0